In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/home/joshua-siraj/Documents/CDI/AutoML/data/RADCURE_challenge_clinical.csv')


In [2]:
from AutoML.analyzer import Analyzer

data.drop(columns=["Study ID", "split"], inplace=True)
analyzer = Analyzer(data, target_variable='death', output_dir='./bruh')

analyzer.run(one_hot=True)

/home/joshua-siraj/Documents/CDI/AutoML/src/AutoML/analyzer.py:171: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.data[col] = pd.to_datetime(self.data[col])
/home/joshua-siraj/Documents/CDI/AutoML/src/AutoML/analyzer.py:171: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.data[col] = pd.to_datetime(self.data[col])
/home/joshua-siraj/Documents/CDI/AutoML/src/AutoML/analyzer.py:171: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.data[col] = pd.to_datetime(self.data[col])
/home/joshua-siraj/Documents/CDI/AutoML/src/AutoML/analyzer.py:171: UserWarning: Could not inf

Config file not found, used a heuristic to define categorical and continuous columns. Please review!
Categorical: ['Stage', 'T Stage', 'Disease Site', 'N Stage', 'Sex', 'EGFRI', 'death', 'Chemotherapy', 'HPV Combined', 'Dose']

Continuous: ['survival_time', 'age at dx']

Config file not found, checking for outliers in categorical variables...
  - Outliers found in Stage: ['IV: 6 out of 2549', 'IIIC: 2 out of 2549', 'IIIA: 2 out of 2549', 'IIA: 1 out of 2549']
Applying changes from config file...

╒══════════════════════════╤═══════════════════╤═══════════╤═════════════╕
│                          │                   │ Missing   │ Overall     │
╞══════════════════════════╪═══════════════════╪═══════════╪═════════════╡
│ n                        │                   │           │ 2552        │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ survival_time, mean (SD) │                   │ 0         │ 4.8 (2.8)   │
├──────────────────────────┼───────────────────┼

/home/joshua-siraj/Documents/CDI/AutoML/.pixi/envs/default/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis
from sksurv.svm import FastSurvivalSVM
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

# Load and prepare dataset
data = pd.read_csv('/home/joshua-siraj/Documents/CDI/AutoML/tutorials/notebooks/bruh/updated_data.csv')
data["event"] = data["death"] == 1
data.rename(columns={'survival_time': 'time'}, inplace=True)
X = data.drop(columns=["time", "death", "event"])
y = Surv.from_dataframe("event", "time", data)

def reduce_features(X_train, X_test, method="variance", threshold=0.01, n_components=5):
    """
    Reduces features using the specified method.
    Supported methods: 'variance', 'pca'
    """
    if method == "variance":
        print(f"Applying Variance Threshold with threshold={threshold}...")
        selector = VarianceThreshold(threshold=threshold)
        X_train_reduced = selector.fit_transform(X_train)
        X_test_reduced = selector.transform(X_test)
        print(f"Reduced from {X_train.shape[1]} to {X_train_reduced.shape[1]} features.")
    elif method == "pca":
        print(f"Applying PCA to retain {n_components} components...")
        pca = PCA(n_components=n_components, random_state=42)
        X_train_reduced = pca.fit_transform(X_train)
        X_test_reduced = pca.transform(X_test)
        print(f"Reduced to {n_components} principal components.")
    else:
        raise ValueError("Unsupported feature reduction method. Use 'variance' or 'pca'.")
    
    return X_train_reduced, X_test_reduced

def train_and_evaluate_models(X, y, feature_reduction=None, reduction_params=None):
    """
    Trains survival models with optional feature reduction and evaluates them.
    """
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Apply feature reduction if specified
    if feature_reduction:
        reduction_params = reduction_params or {}
        X_train, X_test = reduce_features(X_train, X_test, method=feature_reduction, **reduction_params)

    # Initialize models with predefined hyperparameters
    models = {
        "CoxPH": CoxPHSurvivalAnalysis(),
        "RandomForest": RandomSurvivalForest(n_estimators=100, max_depth=5, random_state=42),
        "SVM": FastSurvivalSVM(alpha=1e-4, random_state=42),
        "GradientBoosting": GradientBoostingSurvivalAnalysis(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    }

    results = {}

    for name, model in models.items():
        print(f"Training {name}...")
        # Train the model
        model.fit(X_train, y_train)

        # Predict survival scores
        scores = model.predict(X_test)

        # Evaluate metrics
        c_index = concordance_index_censored(y_test["event"], y_test["time"], scores)[0]

        results[name] = {
            "Concordance Index": c_index,
        }

    return results

# Example usage
# Variance Threshold
print("Feature Reduction: Variance Threshold")
results_var = train_and_evaluate_models(X, y, feature_reduction="variance", reduction_params={"threshold": 0.01})
print(results_var)

# PCA
print("\nFeature Reduction: PCA")
results_pca = train_and_evaluate_models(X, y, feature_reduction="pca", reduction_params={"n_components": 5})
print(results_pca)


Feature Reduction: Variance Threshold
Applying Variance Threshold with threshold=0.01...
Reduced from 61 to 46 features.
Training CoxPH...
Training RandomForest...


/home/joshua-siraj/Documents/CDI/AutoML/.pixi/envs/default/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=2.44578e-24): result may not be accurate.
  delta = solve(
/home/joshua-siraj/Documents/CDI/AutoML/.pixi/envs/default/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=2.84301e-27): result may not be accurate.
  delta = solve(
/home/joshua-siraj/Documents/CDI/AutoML/.pixi/envs/default/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=6.64561e-29): result may not be accurate.
  delta = solve(
/home/joshua-siraj/Documents/CDI/AutoML/.pixi/envs/default/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.19286e-39): result may not be accurate.
  delta = solve(
/home/joshua-siraj/Documents/CDI/AutoML/.pixi/envs/default/lib/python3.11/site-packages/sksurv/l

Training SVM...
Training GradientBoosting...
{'CoxPH': {'Concordance Index': 0.7857343350119707}, 'RandomForest': {'Concordance Index': 0.7673491290349211}, 'SVM': {'Concordance Index': 0.5695533724098076}, 'GradientBoosting': {'Concordance Index': 0.7783620903161892}}

Feature Reduction: PCA
Applying PCA to retain 5 components...
Reduced to 5 principal components.
Training CoxPH...
Training RandomForest...
Training SVM...
Training GradientBoosting...
{'CoxPH': {'Concordance Index': 0.6610418558573433}, 'RandomForest': {'Concordance Index': 0.6883183356724181}, 'SVM': {'Concordance Index': 0.6651366300668703}, 'GradientBoosting': {'Concordance Index': 0.6974242549327169}}
